# Análisis Estadístico de Experimentos

## 1. Análisis de Rendimiento del Crawler (Throughput)

En esta sección, se analiza el rendimiento de los crawlers secuencial y paralelo para determinar si existen diferencias significativas en sus tiempos de ejecución.

### 1.1. Carga de Datos

In [ ]:
import pandas as pd
from scipy.stats import shapiro, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar los datos
df_crawler = pd.read_csv('../crawler_metrics_duckduckgo_20250625_002317.csv')
df_crawler.head()

### 1.2. Test de Normalidad

Se realiza un test de Shapiro-Wilk para determinar si los datos de tiempo de ejecución (`seq_time` y `par_time`) siguen una distribución normal. La hipótesis nula (H0) es que los datos provienen de una distribución normal.

In [ ]:
# Extraer las columnas de tiempo
seq_time = df_crawler['seq_time']
par_time = df_crawler['par_time']

# Realizar el test de Shapiro-Wilk
stat_seq, p_seq = shapiro(seq_time)
stat_par, p_par = shapiro(par_time)

print(f"Test de Shapiro-Wilk para 'seq_time': p-value = {p_seq}")
print(f"Test de Shapiro-Wilk para 'par_time': p-value = {p_par}")

# Visualización de las distribuciones
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(seq_time, kde=True)
plt.title('Distribución de seq_time')

plt.subplot(1, 2, 2)
sns.histplot(par_time, kde=True)
plt.title('Distribución de par_time')
plt.show()

alpha = 0.05
if p_seq > alpha and p_par > alpha:
    print('\nAmbas muestras parecen seguir una distribución normal (no se puede rechazar H0).')
else:
    print('\nAl menos una de las muestras no sigue una distribución normal. Se debería considerar un test no paramétrico.')

### 1.3. Test de Hipótesis

Dado que los datos parecen normales, se procede con un test t de Student para muestras independientes (específicamente, el test de Welch, que no asume varianzas iguales) para comparar las medias de `seq_time` y `par_time`.

*   **Hipótesis Nula (H0):** No hay diferencia entre las medias de los tiempos de ejecución (`mean(seq_time) = mean(par_time)`).
*   **Hipótesis Alternativa (H1):** Existe una diferencia significativa entre las medias (`mean(seq_time) != mean(par_time)`).

In [ ]:
# Solo proceder si los datos son normales
if p_seq > alpha and p_par > alpha:
    # Realizar el t-test de Welch
    stat_ttest, p_ttest = ttest_ind(seq_time, par_time, equal_var=False)

    print(f"Test t de Student (Welch): p-value = {p_ttest}")

    if p_ttest < alpha:
        print('Se rechaza la hipótesis nula: Hay una diferencia estadísticamente significativa entre los tiempos de ejecución.')
        if par_time.mean() < seq_time.mean():
            print('Conclusión: El crawler paralelo es significativamente más eficiente que el secuencial.')
        else:
            print('Conclusión: El crawler secuencial es significativamente más eficiente que el paralelo.')
    else:
        print('No se puede rechazar la hipótesis nula: No hay evidencia de una diferencia significativa entre los tiempos de ejecución.')
else:
    print('No se realiza el t-test porque los datos no cumplen el supuesto de normalidad.')

## 2. Próximos Análisis

Esta sección está reservada para futuros análisis de otros componentes del sistema.